In [1]:
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np
import datetime as dt

# Reading the Datasets

In [2]:
df = pd.read_csv("CLV.csv")

In [3]:
df.head()

,NUMBER_OF_INVOICE,QUANTITY,DATE_OF_INVOICE,PRICE_OF_UNIT,ID_OF_CUSTOMER,COUNTRY
0,536365,6,01/12/2018 04:14,2.55,17850.0,United Kingdom
1,536365,6,01/12/2018 04:14,3.39,17850.0,United Kingdom
2,536365,8,01/12/2018 04:14,2.75,17850.0,United Kingdom
3,536365,6,01/12/2018 04:14,3.39,17850.0,United Kingdom
4,536365,6,01/12/2018 04:14,3.39,17850.0,United Kingdom


# Explore the data — validation and new variables

In [4]:
df1 = df

In [5]:
df1.COUNTRY.nunique()

38

In [6]:
df1['COUNTRY'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [7]:
customer_country=df1[['COUNTRY','ID_OF_CUSTOMER']].drop_duplicates()
customer_country.groupby(['COUNTRY'])['ID_OF_CUSTOMER'].aggregate('count').reset_index().sort_values('ID_OF_CUSTOMER', ascending=False)

,COUNTRY,ID_OF_CUSTOMER
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


In [8]:
df1 = df1.loc[df1['COUNTRY'] == 'United Kingdom']

In [9]:
df1.isnull().sum(axis=0)

NUMBER_OF_INVOICE         0
QUANTITY                  0
DATE_OF_INVOICE           0
PRICE_OF_UNIT             0
ID_OF_CUSTOMER       133600
COUNTRY                   0
dtype: int64

In [10]:
df1 = df1[pd.notnull(df1['ID_OF_CUSTOMER'])]

In [11]:
df1.isnull().sum(axis=0)

NUMBER_OF_INVOICE    0
QUANTITY             0
DATE_OF_INVOICE      0
PRICE_OF_UNIT        0
ID_OF_CUSTOMER       0
COUNTRY              0
dtype: int64

In [12]:
df1['QUANTITY'].min()

-80995

In [13]:
df1 = df1[(df1['QUANTITY']>0)]
df1.shape
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   NUMBER_OF_INVOICE  354345 non-null  object 
 1   QUANTITY           354345 non-null  int64  
 2   DATE_OF_INVOICE    354345 non-null  object 
 3   PRICE_OF_UNIT      354345 non-null  float64
 4   ID_OF_CUSTOMER     354345 non-null  float64
 5   COUNTRY            354345 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 18.9+ MB


In [14]:
def unique_counts(df1):
    for i in df1.columns:
        count = df1[i].nunique()
        print(i, ": ", count)
unique_counts(df1)

NUMBER_OF_INVOICE :  16649
QUANTITY :  294
DATE_OF_INVOICE :  15615
PRICE_OF_UNIT :  403
ID_OF_CUSTOMER :  3921
COUNTRY :  1


In [15]:
df1['TOTAL_PRICE'] = df1['QUANTITY'] * df1['PRICE_OF_UNIT']

In [16]:
df1['DATE_OF_INVOICE'].min()

'01/02/2019 04:11'

In [17]:
NOW = dt.datetime(2020,2,1)
df1['DATE_OF_INVOICE'] = pd.to_datetime(df1['DATE_OF_INVOICE'])

# RFM Customer Segmentation

## Create a RFM table

In [18]:
RFMTable = df1.groupby('ID_OF_CUSTOMER').agg({'DATE_OF_INVOICE': lambda x: (NOW - x.max()).days, 'NUMBER_OF_INVOICE': lambda x: len(x), 'TOTAL_PRICE': lambda x: x.sum()})
RFMTable['DATE_OF_INVOICE'] = RFMTable['DATE_OF_INVOICE'].astype(int)
RFMTable.rename(columns={'DATE_OF_INVOICE': 'RECENCY', 
                         'NUMBER_OF_INVOICE': 'FREQUENCY', 
                         'TOTAL_PRICE': 'MONETARY'}, inplace=True)

## Calculate RFM metrics for each customer

In [19]:
RFMTable

,RECENCY,FREQUENCY,MONETARY
ID_OF_CUSTOMER,,,
12346.0,378,1,77183.60
12747.0,75,103,4196.01
12748.0,57,4596,33719.73
12749.0,75,199,4090.88
12820.0,97,59,942.34
...,...,...,...
18280.0,212,10,180.60
18281.0,56,7,80.82
18282.0,268,12,178.05


In [20]:
RFMTable['RECENCY'][RFMTable.index[0]]

378

In [21]:
print("CustomerID:", int(RFMTable.index[0]), "has recency:", RFMTable['RECENCY'][RFMTable.index[0]],"days, frequency:", RFMTable['FREQUENCY'][RFMTable.index[0]], "trasactions, and monetary value of $", round(RFMTable['MONETARY'][RFMTable.index[0]],2))
print("CustomerID:", int(RFMTable.index[1]), "has recency:", RFMTable['RECENCY'][RFMTable.index[1]],"days, frequency:", RFMTable['FREQUENCY'][RFMTable.index[1]], "trasactions, and monetary value of $", round(RFMTable['MONETARY'][RFMTable.index[1]],2))

CustomerID: 12346 has recency: 378 days, frequency: 1 trasactions, and monetary value of $ 77183.6
CustomerID: 12747 has recency: 75 days, frequency: 103 trasactions, and monetary value of $ 4196.01


In [22]:
first_customer = df1[df1['ID_OF_CUSTOMER']==12346.0]
first_customer 

,NUMBER_OF_INVOICE,QUANTITY,DATE_OF_INVOICE,PRICE_OF_UNIT,ID_OF_CUSTOMER,COUNTRY,TOTAL_PRICE
61619,541431,74215,2019-01-18 05:49:00,1.04,12346.0,United Kingdom,77183.6


## Split the metrics

In [23]:
quantiles = RFMTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

## Create a segmented RFM table

In [24]:
segmented_RFM = RFMTable

In [25]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

## Add segment numbers to the newly created segmented RFM table

In [26]:
segmented_RFM['R_QUARTILE'] = segmented_RFM['RECENCY'].apply(RScore, args=('RECENCY',quantiles,))
segmented_RFM['F_QUARTILE'] = segmented_RFM['FREQUENCY'].apply(FMScore, args=('FREQUENCY',quantiles,))
segmented_RFM['M_QUARTILE'] = segmented_RFM['MONETARY'].apply(FMScore, args=('MONETARY',quantiles,))
segmented_RFM.head()

,RECENCY,FREQUENCY,MONETARY,R_QUARTILE,F_QUARTILE,M_QUARTILE
ID_OF_CUSTOMER,,,,,,
12346.0,378,1,77183.60,4,4,1
12747.0,75,103,4196.01,2,1,1
12748.0,57,4596,33719.73,1,1,1
12749.0,75,199,4090.88,2,1,1
12820.0,97,59,942.34,2,2,2


## Add a new column to combine RFM score: 111 is the highest score

In [27]:
segmented_RFM['RFMScore']=segmented_RFM.R_QUARTILE.map(str)+segmented_RFM.F_QUARTILE.map(str)+\
segmented_RFM.M_QUARTILE.map(str)
segmented_RFM.head()

,RECENCY,FREQUENCY,MONETARY,R_QUARTILE,F_QUARTILE,M_QUARTILE,RFMScore
ID_OF_CUSTOMER,,,,,,,
12346.0,378,1,77183.60,4,4,1,441
12747.0,75,103,4196.01,2,1,1,211
12748.0,57,4596,33719.73,1,1,1,111
12749.0,75,199,4090.88,2,1,1,211
12820.0,97,59,942.34,2,2,2,222


## Who are the top 10 of our best customers!

In [28]:
segmented_RFM[segmented_RFM['RFMScore']=='111'].sort_values('MONETARY', ascending=False).head(10)

,RECENCY,FREQUENCY,MONETARY,R_QUARTILE,F_QUARTILE,M_QUARTILE,RFMScore
ID_OF_CUSTOMER,,,,,,,
18102.0,64,431,259657.30,1,1,1,111
17450.0,55,337,194550.79,1,1,1,111
17511.0,58,963,91062.38,1,1,1,111
16684.0,64,277,66653.56,1,1,1,111
14096.0,64,5111,65164.79,1,1,1,111
15311.0,52,2379,60767.90,1,1,1,111
13089.0,58,1818,58825.83,1,1,1,111
15061.0,57,403,54534.14,1,1,1,111
14088.0,63,589,50491.81,1,1,1,111
